# Cloning CORDS repository

In [ ]:
!git clone https://github.com/decile-team/cords.git
%cd cords/
%ls

Cloning into 'cords'...
remote: Enumerating objects: 3078, done.
remote: Counting objects: 100% (1700/1700), done.
remote: Compressing objects: 100% (782/782), done.
remote: Total 3078 (delta 1087), reused 1466 (delta 898), pack-reused 1378
Receiving objects: 100% (3078/3078), 52.84 MiB | 17.36 MiB/s, done.
Resolving deltas: 100% (1824/1824), done.
/content/cords
benchmarks/  examples/       requirements/        run_ssl.py   train_ssl.py
configs/     LICENSE.txt     requirements.txt     setup.py
cords/       paramtuning.py  run_param_tuning.py  tests/
docs/        README.md       run_sl.py            train_sl.py


# Install prerequisite libraries of CORDS

In [ ]:
!pip install dotmap
!pip install apricot-select
!pip install ray[default]
!pip install ray[tune]

     |████████████████████████████████| 154 kB 5.5 MB/s 
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48787 sha256=e8933f95e093effcc31c07758debf5144a5193270b434344798086a5c4000a8d
  Stored in directory: /root/.cache/pip/wheels/1d/b0/5d/41bab30f23d17864700963dad70bbeda159a409e94f0778f2f
Successfully built apricot-select
     |████████████████████████████████| 54.0 MB 89 kB/s 
     |████████████████████████████████| 72 kB 628 kB/s 
     |████████████████████████████████| 128 kB 58.4 MB/s 
     |████████████████████████████████| 201 kB 56.7 MB/s 
     |████████████████████████████████| 3.2 MB 55.2 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 1.3 MB 40.4 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 160 kB 61.3 MB/s 
     |████████████████████████████████| 271 kB 58.3 MB/s 
  Creat

#Import necessary libraries

In [ ]:
import time
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from cords.utils.data.datasets.SL import gen_dataset
from torch.utils.data import Subset
from cords.utils.config_utils import load_config_data
import os.path as osp
from cords.utils.data.data_utils import WeightedSubset
from ray import tune

#Loading the CIFAR10 dataset

Since CIFAR10 dataset is a predefined dataset in CORDS repository. You can use the gen_dataset function for loading the CIFAR10 dataset.

**Input parameters of gen_dataset function:**

***datadir :*** Directory containing the data. If data is not downloaded, then data will be automatically downloaded into the mentioned directory path.

***dset_name :*** Dataset Name

***feature :*** If "classimb", we make the dataset inherently imbalanced.
          If "classimb", we make the dataset labels noisy.
          If None, we return the standard datasets.

***isnumpy :*** If True, return dataset in the numpy array format.
          If False, return dataset in torch dataset format.




In [ ]:
trainset, validset, testset, num_cls = gen_dataset('data/', 'cifar10', None, isnumpy=False)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


# Create dataloaders

In [ ]:
trn_batch_size = 20
val_batch_size = 20
tst_batch_size = 1000

# Creating the Data Loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trn_batch_size,
                                          shuffle=False, pin_memory=True)

valloader = torch.utils.data.DataLoader(validset, batch_size=val_batch_size,
                                        shuffle=False, pin_memory=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=tst_batch_size,
                                          shuffle=False, pin_memory=True)


#Defining Model

CORDS has a set of predefined models bulit in utils folder. You can import them directly.

In [ ]:
from cords.utils.models import ResNet18
numclasses = 10
device = 'cuda' #Device Argument
model = ResNet18(10)
model = model.to(device)

# Defining Loss Functions

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion_nored = nn.CrossEntropyLoss(reduction='none')

# Checkpointing Utility functions

In [ ]:
def save_ckpt(state, ckpt_path):
    torch.save(state, ckpt_path)


def load_ckpt(ckpt_path, model, optimizer):
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    loss = checkpoint['loss']
    metrics = checkpoint['metrics']
    return start_epoch, model, optimizer, loss, metrics


# Cumulative time calculation

In [ ]:
def generate_cumulative_timing(mod_timing):
    tmp = 0
    mod_cum_timing = np.zeros(len(mod_timing))
    for i in range(len(mod_timing)):
        tmp += mod_timing[i]
        mod_cum_timing[i] = tmp
    return mod_cum_timing / 3600


# Defining Optimizers and schedulers

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-2,
                                  momentum=0.9,
                                  weight_decay=5e-4,
                                  nesterov=False)

#T_max is the maximum number of scheduler steps. Here we are using the number of epochs as the maximum number of scheduler steps.

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                       T_max=300) 


#Get logger object for logging

In [ ]:
def __get_logger(results_dir):
  os.makedirs(results_dir, exist_ok=True)
  # setup logger
  plain_formatter = logging.Formatter("[%(asctime)s] %(name)s %(levelname)s: %(message)s",
                                      datefmt="%m/%d %H:%M:%S")
  logger = logging.getLogger(__name__)
  logger.setLevel(logging.INFO)
  s_handler = logging.StreamHandler(stream=sys.stdout)
  s_handler.setFormatter(plain_formatter)
  s_handler.setLevel(logging.INFO)
  logger.addHandler(s_handler)
  f_handler = logging.FileHandler(os.path.join(results_dir, "results.log"))
  f_handler.setFormatter(plain_formatter)
  f_handler.setLevel(logging.DEBUG)
  logger.addHandler(f_handler)
  logger.propagate = False
  return logger



#Instantiating logger file for logging the information

In [ ]:
import logging
import os
import os.path as osp
import sys

#Results logging directory
results_dir = osp.abspath(osp.expanduser('results'))
logger = __get_logger(results_dir)

In [ ]:
logger.info("hello")

[10/19 19:28:17] __main__ INFO: hello


# Instantiating GLISTER subset selection dataloaders
We instantiate subset dataloaders that can be used for training the models with adaptive subsets.

Each subset dataloader needs data selection strategy arguments in the form of a dotmap dictionary, logger and dataloader specific arguments like batch size, shuffle etc.

We are instantiating GLISTER dataloader here with no warm start. But any dataloader can be instantiated in the same way by passing the required arguments



In [ ]:
from cords.utils.data.dataloader.SL.adaptive import GLISTERDataLoader, OLRandomDataLoader, \
    CRAIGDataLoader, GradMatchDataLoader, RandomDataLoader
from dotmap import DotMap

selection_strategy = 'GLISTER'
dss_args = dict(model=model,
                loss=criterion_nored,
                eta=0.01,
                num_classes=10,
                num_epochs=300,
                device='cuda',
                fraction=0.1,
                select_every=20,
                kappa=0,
                linear_layer=False,
                selection_type='SL',
                greedy='Stochastic')
dss_args = DotMap(dss_args)

dataloader = GLISTERDataLoader(trainloader, valloader, dss_args, logger, 
                                  batch_size=20, 
                                  shuffle=True,
                                  pin_memory=False)



/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


# Additional arguments for training, evaluation and checkpointing

In [ ]:
#Training Arguments
num_epochs = 300

#Arguments for results logging
print_every = 1
print_args = ["val_loss", "val_acc", "tst_loss", "tst_acc", "time"]

#Argumets for checkpointing
save_every = 20
is_save = True

#Evaluation Metrics
trn_losses = list()
val_losses = list()
tst_losses = list()
subtrn_losses = list()
timing = list()
trn_acc = list()
val_acc = list()  
tst_acc = list()  
subtrn_acc = list()


#Custom Training loop with evaluation

Subset dataloader returns data samples, labels and associated weights with each data sample. Hence, inorder to incorporate the weights in the dataloader into the training loop, we use a **loss function**  with **reduction='none'** to get per-sample loss values. Then we calculate the weighted average of batch losses using the following code snippet:

`loss = torch.dot(losses, weights/(weights.sum()))`

---
**NOTE**

### If you want to implement a custom training loop, please note that the subset dataloaders also returns additional weight parameter for each data sample.
---

In [ ]:
"""
################################################# Training Loop #################################################
"""
for epoch in range(num_epochs):
    subtrn_loss = 0
    subtrn_correct = 0
    subtrn_total = 0
    model.train()
    start_time = time.time()
    for _, (inputs, targets, weights) in enumerate(dataloader):
        inputs = inputs.to(device)
        targets = targets.to(device, non_blocking=True)
        weights = weights.to(device)  
        optimizer.zero_grad()
        outputs = model(inputs)
        losses = criterion_nored(outputs, targets)
        loss = torch.dot(losses, weights/(weights.sum()))
        loss.backward()
        subtrn_loss += loss.item()
        optimizer.step()
        _, predicted = outputs.max(1)
        subtrn_total += targets.size(0)
        subtrn_correct += predicted.eq(targets).sum().item()
    epoch_time = time.time() - start_time
    scheduler.step()
    timing.append(epoch_time)


    """
    ################################################# Evaluation Loop #################################################
    """

    if (epoch + 1) % print_every == 0:
        trn_loss = 0
        trn_correct = 0
        trn_total = 0
        val_loss = 0
        val_correct = 0
        val_total = 0
        tst_correct = 0
        tst_total = 0
        tst_loss = 0
        model.eval()

        if ("trn_loss" in print_args) or ("trn_acc" in print_args):
            with torch.no_grad():
                for _, (inputs, targets) in enumerate(trainloader):
                    inputs, targets = inputs.to(device), \
                                      targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    trn_loss += loss.item()
                    if "trn_acc" in print_args:
                        _, predicted = outputs.max(1)
                        trn_total += targets.size(0)
                        trn_correct += predicted.eq(targets).sum().item()
                trn_losses.append(trn_loss)

            if "trn_acc" in print_args:
                trn_acc.append(trn_correct / trn_total)

        if ("val_loss" in print_args) or ("val_acc" in print_args):
            with torch.no_grad():
                for _, (inputs, targets) in enumerate(valloader):
                    inputs, targets = inputs.to(device), \
                                      targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    val_loss += loss.item()
                    if "val_acc" in print_args:
                        _, predicted = outputs.max(1)
                        val_total += targets.size(0)
                        val_correct += predicted.eq(targets).sum().item()
                val_losses.append(val_loss)

            if "val_acc" in print_args:
                val_acc.append(val_correct / val_total)

        if ("tst_loss" in print_args) or ("tst_acc" in print_args):
            with torch.no_grad():
                for _, (inputs, targets) in enumerate(testloader):
                    inputs, targets = inputs.to(device), \
                                      targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    tst_loss += loss.item()
                    if "tst_acc" in print_args:
                        _, predicted = outputs.max(1)
                        tst_total += targets.size(0)
                        tst_correct += predicted.eq(targets).sum().item()
                tst_losses.append(tst_loss)

            if "tst_acc" in print_args:
                tst_acc.append(tst_correct / tst_total)

        if "subtrn_acc" in print_args:
            subtrn_acc.append(subtrn_correct / subtrn_total)

        if "subtrn_losses" in print_args:
            subtrn_losses.append(subtrn_loss)

        print_str = "Epoch: " + str(epoch + 1)

        """
        ################################################# Results Printing #################################################
        """

        for arg in print_args:

            if arg == "val_loss":
                print_str += " , " + "Validation Loss: " + str(val_losses[-1])

            if arg == "val_acc":
                print_str += " , " + "Validation Accuracy: " + str(val_acc[-1])

            if arg == "tst_loss":
                print_str += " , " + "Test Loss: " + str(tst_losses[-1])

            if arg == "tst_acc":
                print_str += " , " + "Test Accuracy: " + str(tst_acc[-1])

            if arg == "trn_loss":
                print_str += " , " + "Training Loss: " + str(trn_losses[-1])

            if arg == "trn_acc":
                print_str += " , " + "Training Accuracy: " + str(trn_acc[-1])

            if arg == "subtrn_loss":
                print_str += " , " + "Subset Loss: " + str(subtrn_losses[-1])

            if arg == "subtrn_acc":
                print_str += " , " + "Subset Accuracy: " + str(subtrn_acc[-1])

            if arg == "time":
                print_str += " , " + "Timing: " + str(timing[-1])

        logger.info(print_str)

    """
    ################################################# Checkpoint Saving #################################################
    """

    if ((epoch + 1) % save_every == 0) and is_save:

        metric_dict = {}

        for arg in print_args:
            if arg == "val_loss":
                metric_dict['val_loss'] = val_losses
            if arg == "val_acc":
                metric_dict['val_acc'] = val_acc
            if arg == "tst_loss":
                metric_dict['tst_loss'] = tst_losses
            if arg == "tst_acc":
                metric_dict['tst_acc'] = tst_acc
            if arg == "trn_loss":
                metric_dict['trn_loss'] = trn_losses
            if arg == "trn_acc":
                metric_dict['trn_acc'] = trn_acc
            if arg == "subtrn_loss":
                metric_dict['subtrn_loss'] = subtrn_losses
            if arg == "subtrn_acc":
                metric_dict['subtrn_acc'] = subtrn_acc
            if arg == "time":
                metric_dict['time'] = timing

        ckpt_state = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': criterion_nored,
            'metrics': metric_dict
        }

        # save checkpoint
        save_ckpt(ckpt_state, 'model.pt')
        logger.info("Model checkpoint saved at epoch: {0:d}".format(epoch + 1))

"""
################################################# Results Summary #################################################
"""

logger.info("{0:s} Selection Run---------------------------------".format(selection_strategy))
logger.info("Final SubsetTrn: {0:f}".format(subtrn_loss))
if "val_loss" in print_args:
    if "val_acc" in print_args:
        logger.info("Validation Loss: %.2f , Validation Accuracy: %.2f", val_loss, val_acc[-1])
    else:
        logger.info("Validation Loss: %.2f", val_loss)

if "tst_loss" in print_args:
    if "tst_acc" in print_args:
        logger.info("Test Loss: %.2f, Test Accuracy: %.2f", tst_loss, tst_acc[-1])
    else:
        logger.info("Test Data Loss: %f", tst_loss)
logger.info('---------------------------------------------------------------------')
logger.info(selection_strategy)
logger.info('---------------------------------------------------------------------')

"""
################################################# Final Results Logging #################################################
"""

if "val_acc" in print_args:
    val_str = "Validation Accuracy, "
    for val in val_acc:
        val_str = val_str + " , " + str(val)
    logger.info(val_str)

if "tst_acc" in print_args:
    tst_str = "Test Accuracy, "
    for tst in tst_acc:
        tst_str = tst_str + " , " + str(tst)
    logger.info(tst_str)

if "time" in print_args:
    time_str = "Time, "
    for t in timing:
        time_str = time_str + " , " + str(t)
    logger.info(timing)

timing_array = np.array(timing)
cum_timing = list(generate_cumulative_timing(timing_array))
logger.info("Total time taken by %s = %.4f ", selection_strategy, cum_timing[-1])




[10/19 19:28:35] __main__ INFO: Epoch: 1 , Validation Loss: 492.08510732650757 , Validation Accuracy: 0.2628 , Test Loss: 19.398529291152954 , Test Accuracy: 0.2813 , Timing: 7.384371995925903
[10/19 19:28:50] __main__ INFO: Epoch: 2 , Validation Loss: 462.1935260295868 , Validation Accuracy: 0.3188 , Test Loss: 17.76013433933258 , Test Accuracy: 0.3481 , Timing: 7.13740348815918
[10/19 19:29:05] __main__ INFO: Epoch: 3 , Validation Loss: 406.5821268558502 , Validation Accuracy: 0.392 , Test Loss: 15.817896723747253 , Test Accuracy: 0.4154 , Timing: 7.385777473449707
[10/19 19:29:20] __main__ INFO: Epoch: 4 , Validation Loss: 384.15974444150925 , Validation Accuracy: 0.4114 , Test Loss: 15.2018541097641 , Test Accuracy: 0.4156 , Timing: 7.098953723907471
[10/19 19:29:34] __main__ INFO: Epoch: 5 , Validation Loss: 380.91055899858475 , Validation Accuracy: 0.4478 , Test Loss: 14.655442833900452 , Test Accuracy: 0.4532 , Timing: 7.023574590682983
[10/19 19:29:49] __main__ INFO: Epoch: 6 ,